In [1]:
import pandas as pd


import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [72]:
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [73]:


missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head(10)



,count,%
G/SoT,185.0,26.241135
Guaranteed Compensation,181.0,25.673759
name,181.0,25.673759
Unnamed: 0_y,181.0,25.673759
First Name,181.0,25.673759
Last Name,181.0,25.673759
Club,181.0,25.673759
Base Salary,181.0,25.673759
G/Sh,116.0,16.453901
npxG/Sh,116.0,16.453901


In [46]:
df['Club'].value_counts()

Toronto FC                26
Columbus Crew             23
DC United                 23
LAFC                      22
Seattle Sounders FC       21
Vancouver Whitecaps       21
FC Cincinnati             21
Colorado Rapids           21
Philadelphia Union        21
Minnesota United          21
Real Salt Lake            20
Montreal                  20
Atlanta United            20
Portland Timbers          20
Inter Miami               19
Houston Dynamo            19
FC Dallas                 19
LA Galaxy                 19
New England Revolution    18
Nashville SC              18
Austin FC                 18
New York Red Bulls        18
San Jose Earthquakes      17
New York City FC          15
Orlando City SC           15
Chicago Fire              14
Sporting Kansas City      14
Major League Soccer        1
Name: Club, dtype: int64

In [74]:
df.drop(columns=['Unnamed: 0_y', 'First Name', 'Last Name', 'name', '_merge','Club','Guaranteed Compensation'],inplace=True)

In [48]:
# df['Club'].fillna('Major League Soccer',inplace=True)

# ## dont need club for NMF 

In [49]:
df.shape

(705, 81)

In [50]:
df.columns

Index(['fb_id', 'player_name', 'Squad_from_op', 'Opposing_Poss',
       'Opposing_Touches', 'Nation', 'Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct', 'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
       'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
       'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
       'PK_won', 'Aerial_win_pct', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
       'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG'

In [51]:
df['Base Salary'].sample(5)

176    450000.0
646    250000.0
258    425000.0
616         NaN
505         NaN
Name: Base Salary, dtype: float64

In [58]:
df[['G/SoT','SoT%','npxG/Sh','Dist']].describe()

,G/SoT,SoT%,npxG/Sh,Dist
count,0,0,0,0
unique,0,0,0,0
top,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN


In [75]:
cols = ['G/Sh','G/SoT','SoT%','npxG/Sh','Dist','Base Salary']

In [69]:

df[['G/Sh','G/SoT','SoT%','npxG/Sh','Dist','Base Salary']].describe()

,G/SoT,SoT%,npxG/Sh,Dist,Base Salary
count,705.000000,705.000000,705.000000,705.000000,7.050000e+02
mean,0.027278,30.385059,0.013745,2.395677,3.811160e+05
std,0.067275,17.542877,0.026658,3.205736,4.174811e+05
min,0.000000,0.000000,0.000398,0.214706,6.354700e+04
25%,0.004059,21.400000,0.003540,0.792079,1.150000e+05
50%,0.020000,30.385059,0.007692,1.442105,3.750000e+05
75%,0.027278,37.800000,0.013745,2.395677,3.811160e+05
max,1.000000,100.000000,0.400000,26.727273,4.500000e+06


In [76]:
for col in cols:
    print(df[col].mean())
    df[col] = df[col].fillna(df[col].mean())

missing = [df.isnull().sum(),100*df.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

0.009939746728555306
0.027277585423982696
30.38505942275043
0.013745106122472362
2.395677172663176
381115.9694656489


,count,%
fb_id,0.0,0.0
carries_att_3rd,0.0,0.0
G/Sh,0.0,0.0
SoT/90,0.0,0.0
Sh/90,0.0,0.0


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 81 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fb_id                       705 non-null    object 
 1   player_name                 705 non-null    object 
 2   Squad_from_op               705 non-null    object 
 3   Opposing_Poss               705 non-null    float64
 4   Opposing_Touches            705 non-null    float64
 5   Nation                      705 non-null    object 
 6   Position                    705 non-null    object 
 7   Nineties                    705 non-null    float64
 8   xA                          705 non-null    float64
 9   Key_pass                    705 non-null    float64
 10  Comp_prog                   705 non-null    float64
 11  Total_Cmp                   705 non-null    float64
 12  Total_Att                   705 non-null    float64
 13  Total_Cmp_pct               705 non

In [82]:
defs = df.loc[df['Position']=='DF']

mini = defs[['player_name', 'Squad_from_op', 'Opposing_Poss',
       'Opposing_Touches', 'Nation', 'Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct']]

In [112]:
# Create a MaxAbsScaler: scaler
scaler = MaxAbsScaler()

# Create an NMF model: nmf
nmf = NMF(5,max_iter=200)

# Create a Normalizer: normalizer
normalizer = Normalizer()

# Create a pipeline: pipeline
pipeline = make_pipeline(scaler,nmf,normalizer)

# Apply fit_transform to playesr: norm_features
norm_features = pipeline.fit_transform(mini.select_dtypes(float))

/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/Users/robertrustia/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [113]:
normed_df = pd.DataFrame(norm_features,index=mini.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227 entries, Julian Gressel to Lalas Abubakar
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       227 non-null    float64
 1   1       227 non-null    float64
 2   2       227 non-null    float64
 3   3       227 non-null    float64
 4   4       227 non-null    float64
dtypes: float64(5)
memory usage: 10.6+ KB


In [114]:
normed_df[:15]

,0,1,2,3,4
Julian Gressel,0.000000,0.882848,0.117556,0.441582,0.108470
Brooks Lennon,0.068180,0.867715,0.255916,0.390277,0.156888
Kai Wagner,0.367178,0.754873,0.228585,0.477313,0.123566
Daniel Lovitz,0.253203,0.757942,0.353322,0.432120,0.223266
Žan Kolmanič,0.189046,0.881080,0.186638,0.324978,0.217981
Aaron Herrera,0.268825,0.806231,0.197749,0.423429,0.243573
Romain Métanire,0.215341,0.848008,0.161504,0.397883,0.223866
Guðmundur Þórarinsson,0.194288,0.902553,0.199461,0.287491,0.158788
DeJuan Jones,0.172271,0.888485,0.089669,0.355085,0.216314
Graham Zusi,0.579843,0.722502,0.005137,0.335213,0.171406


In [115]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson     1.000000
D.J. Taylor        0.997575
Mustafa Kizza      0.995366
Steven Moreira     0.988318
Jaylin Lindsey     0.986950
Malte Amundsen     0.984327
Justin Che         0.982883
Noah Powder        0.982226
Chris Duvall       0.981875
Niko Hämäläinen    0.980531
dtype: float64

In [116]:
tb = normed_df.loc['Tanner Beason']
sims = normed_df.dot(tb)

sims.nlargest(10)

Tanner Beason        1.000000
Alexander Callens    0.996168
Jhohan Romaña        0.995858
Jorge Figal          0.993946
Teenage Hadebe       0.993892
Christian Makoun     0.992399
Anton Walkes         0.990971
Robin Jansson        0.990686
Julio Cascante       0.990338
Marcelo Silva        0.989581
dtype: float64

In [92]:
defs.loc[defs['Squad_from_op']=='San Jose']

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Nation,Position,Nineties,xA,Key_pass,...,pAdj_Yellow_cards,pAdj_Red_cards,pAdj_Second_yellow,pAdj_Fouls,pAdj_def_interceptions,pAdj_from_mish_tackles_won,pAdj_PK_conceded,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost
184,da0bb55a,Paul Marie,San Jose,47.4,548.264706,fr FRA,DF,11.1,0.090090,1.531532,...,5.258195,0.000000,0.0,3.286372,3.450691,2.793416,0.0,0.000000,15.610267,2.957735
226,9b3ab988,Marcos López,San Jose,47.4,548.264706,pe PER,DF,15.7,0.076433,0.828025,...,3.485229,0.000000,0.0,3.020532,5.924889,1.394092,0.0,0.000000,11.849778,2.555835
257,6c510f2d,Luciano Abecasis,San Jose,47.4,548.264706,ar ARG,DF,13.7,0.116788,0.802920,...,3.727753,0.000000,0.0,4.792826,4.127156,3.594619,0.0,0.000000,16.242354,2.662681
269,5f6c69d8,Tommy Thompson,San Jose,47.4,548.264706,us USA,DF,8.3,0.132530,1.204819,...,4.834530,0.000000,0.0,1.318508,5.493785,2.856768,0.0,0.000000,14.503591,1.758011
415,e97787d9,Oswaldo Alanís,San Jose,47.4,548.264706,mx MEX,DF,22.7,0.008811,0.176211,...,0.241049,0.000000,0.0,1.205244,4.499579,1.526643,0.0,0.000000,13.338038,3.294335
455,d4438745,Nathan Cardoso,San Jose,47.4,548.264706,br BRA,DF,19.2,0.020833,0.156250,...,0.189993,0.094997,0.0,4.084858,8.359709,6.174785,0.0,0.000000,15.294467,6.364778
548,3f1b018e,Tanner Beason,San Jose,47.4,548.264706,us USA,DF,21.4,0.009346,0.046729,...,0.340923,0.000000,0.0,1.022768,3.409227,0.937537,0.0,0.085231,12.614140,6.051378


In [117]:
#I need to look at what features are in each category to figure out what to set p too, other hyperparamters 
#need to be tuned 